In [38]:
!pip install gym_super_mario_bros==7.3.0 nes_py
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
tcmalloc: large alloc 1821458432 bytes == 0x2538000 @  0x7ff3f243f1e7 0x4b2590 0x5ad01c 0x5dcfef 0x58f92b 0x590c33 0x5e48ac 0x4d20fa 0x51041f 0x58fd37 0x50ca37 0x58fd37 0x50ca37 0x5b575e 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b575e 0x4bad0a 0x4d3249
tcmalloc: large alloc 1821458432 bytes == 0x6ee4c000 @  0x7ff3f243f1e7 0x4b2590 0x5ad01c 0x64b047 0x4d22a6 0x51041f 0x58fd37 0x50ca37 0x58fd37 0x50ca37 0x5b575e 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b575e 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6
tcmalloc: large alloc 1821458432 bytes == 0xdb

In [39]:
import io
import base64
import os 
import gym_super_mario_bros

from matplotlib import pyplot as plt
from IPython.display import HTML
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

In [40]:
game_envi = gym_super_mario_bros.make('SuperMarioBros-v3')
game_envi = JoypadSpace(game_envi, SIMPLE_MOVEMENT)

game_envi = GrayScaleObservation(game_envi, keep_dim=True)
game_envi = DummyVecEnv([lambda: game_envi])
game_envi = VecFrameStack(game_envi, 4, channels_order='last')

state = game_envi.reset()
state, reward, done, info = game_envi.step([5])

In [41]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [42]:
CHECKPOINT_DIR = 'checkpoint'
LOG_DIR = 'logs'
CB = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [43]:
model = PPO('CnnPolicy', game_envi, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


Exception ignored in: <function Monitor.__del__ at 0x7f94785b8b90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitor.py", line 289, in __del__
  File "/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitor.py", line 178, in close
  File "/usr/local/lib/python3.7/dist-packages/gym/core.py", line 298, in close
  File "/usr/local/lib/python3.7/dist-packages/gym/core.py", line 298, in close
  File "/usr/local/lib/python3.7/dist-packages/nes_py/nes_env.py", line 346, in close
ValueError: env has already been closed.


In [44]:
model.learn(total_timesteps=50000, callback=CB)

Logging to logs/PPO_5
----------------------------
| time/              |     |
|    fps             | 137 |
|    iterations      | 1   |
|    time_elapsed    | 3   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 70            |
|    iterations           | 2             |
|    time_elapsed         | 14            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 6.1494065e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00155      |
|    learning_rate        | 1e-06         |
|    loss                 | 177           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000113     |
|    value_loss           | 434           |
---------------------------

In [45]:
model.save('DeGuzman_Mario_Final_Model')

In [46]:
state = game_envi.reset()

In [47]:
from gym import wrappers
game_envi = gym_super_mario_bros.make('SuperMarioBros2-v0')
game_envi = wrappers.Monitor(game_envi, "./gym-results", force=True)
game_envi.reset()
for _ in range(4000):
    action = game_envi.action_space.sample()
    state, reward, done, info = game_envi.step(action)
    if done: break
game_envi.close()

/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


In [48]:
video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % game_envi.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))